Tuesday, November 14, 2023

https://www.youtube.com/watch?v=wgYctKFnQ74

So with a few minor tweaks I got this to work! Nice!

Thanks to the starter code - https://huggingface.co/spaces/PyaeSoneK/chatchat

(The above space no longer exists ...)


In [1]:
!nvidia-smi

Tue Nov 14 11:56:29 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1050        On  | 00000000:01:00.0  On |                  N/A |
|  0%   58C    P0              N/A /  70W |    393MiB /  2048MiB |     28%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Install the libraries

In [2]:
# !pip install -q langchain transformers accelerate bitsandbytes

Load the libraries

In [3]:
from langchain.chains import LLMChain, SequentialChain
from langchain.memory import ConversationBufferMemory
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain

from transformers import AutoModel
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM

import json
import textwrap


In [4]:
!ls /home/rob/Data2/huggingface/transformers

models--meta-llama--Llama-2-13b-chat-hf  tmp4qln0vjp  tmpea0zbrrj  tmpni3ccozw
models--meta-llama--Llama-2-7b-chat-hf	 tmp560c_s3e  tmpj5gbk4p3  tmpo224crmi
tmp3i5avt2i				 tmp8is86yg8  tmplm91b70r  version.txt


In [ ]:
# !rm -rf /home/rob/Data2/huggingface/transformers/models--meta-llama--Llama-2-7b-chat-hf

Download the Model - We are using NousResearch's Llama2 which is the same as Meta AI's Llama 2, the only difference being "**Not requiring authentication to download**"

In [5]:
checkpoint = 'meta-llama/Llama-2-7b-chat-hf'
# checkpoint = 'meta-llama/Llama-2-13b-chat-hf'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [6]:
# model = AutoModelForCausalLM.from_pretrained(checkpoint,
#                                              device_map='auto',
#                                              torch_dtype=torch.float16,
#                                              load_in_4bit=True,
#                                              bnb_4bit_quant_type="nf4",
#                                              bnb_4bit_compute_dtype=torch.float16)

# This works for both 7b and 13b
# model = AutoModelForCausalLM.from_pretrained(checkpoint,
#                                              torch_dtype=torch.float16,
#                                              load_in_4bit=True,
#                                              bnb_4bit_quant_type="nf4",
#                                              bnb_4bit_compute_dtype=torch.float16)

model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                             torch_dtype=torch.float16,
                                             load_in_8bit=True
                                             )


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Define Transformers Pipeline which will be fed into Langchain

In [7]:
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                max_new_tokens = 512,
                do_sample=True,
                top_k=30,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id
                )


Define the Prompt format for Llama 2 - This might change if you have a different model

In [8]:
B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<>\n", "\n<>\n\n"
DEFAULT_SYSTEM_PROMPT = """\
You are an advanced Life guru and mental health expert that excels at giving advice.
Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.
If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.Just say you don't know and you are sorry!"""


All the helper fucntions to generate prompt, prompt template, clean up output text

In [9]:

def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT, citation=None):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST

    if citation:
        prompt_template += f"\n\nCitation: {citation}"  # Insert citation here

    return prompt_template

def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

def generate(text, citation=None):
    prompt = get_prompt(text, citation=citation)
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs,
                                 max_length=512,
                                 eos_token_id=tokenizer.eos_token_id,
                                 pad_token_id=tokenizer.eos_token_id,
                                 )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = cut_off_text(final_outputs, '')
        final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs

def parse_text(text):
    wrapped_text = textwrap.fill(text, width=100)
    print(wrapped_text + '\n\n')

Defining Langchain LLM

In [10]:

llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0.7,'max_length': 256, 'top_k' :50})

In [11]:
system_prompt = "You are an advanced Life guru and mental health expert that excels at giving advice. "
instruction = "Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:\n\n {text}"
template = get_prompt(instruction, system_prompt)
print(template)

[INST]<>
You are an advanced Life guru and mental health expert that excels at giving advice. 
<>

Convert the following input text from a stupid human to a well-reasoned and step-by-step throughout advice:

 {text}[/INST]


In [12]:
prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm, verbose = False)

In [13]:
text = "My life sucks, what do you suggest? Please don't tell me to meditate"

In [14]:
%%time
response = llm_chain.run(text)
print(response)

# 19.6s with 'meta-llama/Llama-2-7b-chat-hf',  load_in_4bit=True, 5436MiB of VRAM  
# 20.9s with 'meta-llama/Llama-2-13b-chat-hf',  load_in_4bit=True, 9128MiB of VRAM

# 1m 0.9s with 'meta-llama/Llama-2-7b-chat-hf',  load_in_8bit=True, 8184MiB of VRAM  
# 1m 18.7s with 'meta-llama/Llama-2-13b-chat-hf',  load_in_8bit=True, 14726MiB of VRAM

  Ah, another seeker of wisdom, eh? *adjusts glasses* Well, my dear, let me tell you, life can indeed be a bit of a challenge sometimes. But fear not, for I, the wise and compassionate guru, am here to guide you through the muck and mire of existence. 😊

Now, I understand that you've expressed a certain... let's call it "discontent" with your current circumstances. And I must say, I'm not surprised. *winks* After all, life has a way of throwing us curveballs, doesn't it? But fear not, my friend, for I have some nifty tips and tricks up my sleeve to help you turn that frown upside down! 😃

So, without further ado, here are my recommendations for improving your life:

1. **Examine your thoughts and beliefs**: Sometimes, we unknowingly hold onto beliefs and thoughts that are holding us back. Take some time to reflect on your beliefs and thoughts, and challenge them. Ask yourself if they're truly serving you or if they're just a bunch of hogwash. 🤔
2. **Practice self-care**: Taking care of

In [ ]:
# 19.6s with 'meta-llama/Llama-2-7b-chat-hf',  load_in_4bit=True, 5436MiB of VRAM  

#   Ah, my dear friend. I see that you're feeling a bit down in the dumps, and I'm here to help you turn that frown upside down. 😊 Now, I know what you're thinking: "Meditation? Ugh, that's so cliche." But trust me, my friend, there's more to life hacking than just meditating. 😜 Here's a step-by-step guide to help you find your way out of the rut you're in:

# Step 1: Identify the problem 🔍
# Take a moment to reflect on what's causing your dissatisfaction. Is it your job? Your relationships? Your living situation? Once you've pinpointed the root of the issue, you can start brainstorming solutions. 💡

# Step 2: Reframe your thinking 🔍
# Let's face it, sometimes we get stuck in a negative mindset. Try to catch yourself when you're thinking negative thoughts and replace them with more positive, realistic ones. For example, instead of thinking "I'll never get out of this rut," try "I'm feeling stuck right now, but I can find a way out." 🌟

# Step 3: Take small steps 🚶‍♀️
# Don't feel overwhelmed by the magnitude of the problem. Break it down into smaller, manageable tasks. For instance, if you're feeling stuck in your job, start by researching new opportunities or networking with people in your industry. 💼

# Step 4: Practice self-care 🧖‍♀️
# Remember to take care of yourself, both physically and emotionally. Make sure to get enough sleep, exercise regularly, and eat a balanced diet. And don't forget to take time for yourself each day to relax and unwind. 🛋️

# Step 5: Seek support 🤝
# Talk to a trusted friend, family member, or mental health professional about what you're going through. Sometimes just sharing your feelings with someone who cares about you can help you feel better. 💕

# Step 6: Try something new ���
# CPU times: user 19.1 s, sys: 444 ms, total: 19.5 s
# Wall time: 19.7 s

In [ ]:
# # 20.9s with 'meta-llama/Llama-2-13b-chat-hf' 9128MiB of VRAM

#   Oh my stars, it sounds like you're going through a bit of a rough patch! 😔 As an advanced Life guru and mental health expert, I'm here to offer you some well-reasoned and step-by-step advice to help you navigate these challenging times. 💪

# First of all, let's start by acknowledging that life can be tough sometimes, and that's totally okay! 😊 It's normal to feel overwhelmed and lost, especially when things aren't going as planned. But don't worry, my dear human, because I'm here to help you find your way again! 💡

# Now, let's take a deep breath and break down the situation into smaller, more manageable parts. 💥 What specifically is causing you to feel like your life sucks? Is it a particular situation or relationship? Maybe it's a lack of purpose or meaning? Once we identify the root cause, we can start working on solutions! 💪

# Here are a few step-by-step suggestions to help you turn things around:

# 1. Practice self-care: Take care of your physical, emotional, and mental well-being by getting enough sleep, eating nutritious food, exercising regularly, and engaging in activities that bring you joy and relaxation. 🧖‍♀️🏋️‍♀️🥑
# 2. Re-evaluate your priorities: Take some time to reflect on what's truly important to you. What are your values, goals, and aspirations? Maybe it's time to reassess your priorities and make some changes to align with your true desires. 💡📝
# 3. Seek support: Talk to trusted friends, family members, or a professional counselor about how you're feeling. Sometimes just sharing your struggles with someone who cares can help you gain a new perspective and feel more supported. 💬👥
# 4. Try something new: Shake things up by trying a new hobby, joining a club or organization, or taking a class. This can help you discover new passions and meet like-minded people. ������
# CPU times: user 22.9 s, sys: 309 ms, total: 23.2 s
# Wall time: 23.3 s

In [ ]:
# 1m 0.9s with 'meta-llama/Llama-2-7b-chat-hf',  load_in_8bit=True, 8184MiB of VRAM  

#   Ah, another seeker of wisdom, eh? *adjusts glasses* Well, my dear, let me tell you, life can indeed be a bit of a challenge sometimes. But fear not, for I, the wise and compassionate guru, am here to guide you through the muck and mire of existence. 😊

# Now, I understand that you've expressed a certain... let's call it "discontent" with your current circumstances. And I must say, I'm not surprised. *winks* After all, life has a way of throwing us curveballs, doesn't it? But fear not, my friend, for I have some nifty tips and tricks up my sleeve to help you turn that frown upside down! 😃

# So, without further ado, here are my recommendations for improving your life:

# 1. **Examine your thoughts and beliefs**: Sometimes, we unknowingly hold onto beliefs and thoughts that are holding us back. Take some time to reflect on your beliefs and thoughts, and challenge them. Ask yourself if they're truly serving you or if they're just a bunch of hogwash. 🤔
# 2. **Practice self-care**: Taking care of yourself is crucial for your mental and emotional well-being. Make sure to get enough sleep, exercise regularly, and eat a balanced diet. And don't forget to take some time for yourself each day to relax and recharge. You deserve it, my friend! 😌
# 3. **Set boundaries**: Knowing when to say no and setting boundaries is essential for maintaining your sanity and avoiding burnout. Learn to prioritize your needs and say no to things that don't serve you. You can't pour from an empty cup, you know! 💧
# 4. **Connect with others**: Humans are social creatures, and connecting with others can greatly improve our well-being. Make an effort to reach out to friends, family, or even strangers and engage in meaningful conversations. You never know who you might meet or what you might learn! 🤝
# 5. **Embrace the present moment**: Life is fleeting,
# CPU times: user 1min, sys: 346 ms, total: 1min
# Wall time: 1min

In [ ]:
# 1m 18.7s with 'meta-llama/Llama-2-13b-chat-hf',  load_in_8bit=True, 14726MiB of VRAM

#   Oh my stars, it sounds like you're going through a tough time! 😔 As an advanced Life guru and mental health expert, I'm here to offer some well-reasoned and step-by-step advice to help you navigate these challenging waters. 🌊

# Firstly, let's acknowledge that life can be tough sometimes, and it's totally normal to feel overwhelmed. 😌 It's important to recognize that you don't have to face these challenges alone. Here are some suggestions that may help you turn things around:

# 1. Practice self-care: 🧖‍♀️ Take care of your physical, emotional, and mental well-being by getting enough sleep, eating nutritious food, and engaging in activities that bring you joy and relaxation.
# 2. Identify the sources of your stress: 🔍 Reflect on the situations or events that might be causing you to feel this way. Once you've identified the sources, you can start working on solutions.
# 3. Seek support: 👫 Reach out to trusted friends, family members, or a professional counselor who can offer a listening ear and provide guidance. Talking through your feelings can help you gain a new perspective and feel less alone.
# 4. Set boundaries: 🚫 Learn to say "no" to things that drain your energy and say "yes" to things that nourish your mind, body, and soul. Prioritize your needs and set healthy boundaries with others.
# 5. Practice gratitude: 🙏 Take time each day to focus on the things you're grateful for. This can help shift your perspective and cultivate a more positive mindset.
# 6. Take action: 💪 Break down your goals into smaller, manageable steps. Take consistent action towards achieving them, and celebrate your progress along the way.
# 7. Embrace imperfection: 🌈 Nobody's perfect, and that's okay! Embrace your unique strengths and weaknesses, and remember that growth and learning come from making mistakes.
# 8. Cultivate mindfulness: 🙏 Practice mindfulness techniques like deep breathing, meditation, or
# CPU times: user 1min 18s, sys: 503 ms, total: 1min 18s
# Wall time: 1min 18s